In [ ]:
import numpy as np
import pyvista as pv
from kale.algorithms import contour_banded
from kale import theme, helpers
import xarray as xr

In [ ]:
MESH_GEOMETRY_FILE_NAME = "nankai_121728_clean.vtk"
MESH_VALUES_FILE_NAME = "2023_03_04_00_02_03.hdf"
MESH_Z_SCALE = 0.025
TIME_STEP_INTERESTING = 1000

In [ ]:
ds = xr.open_dataset(MESH_VALUES_FILE_NAME, engine="netcdf4")
data = ds["loading_rate"][TIME_STEP_INTERESTING, :]
mesh = pv.read(MESH_GEOMETRY_FILE_NAME)
mesh.cell_data["data"] = data
mesh.points[:, -1] *= MESH_Z_SCALE

mesh.points[:, 0]

boundary = mesh.extract_feature_edges(
    boundary_edges=True,
    non_manifold_edges=False,
    feature_edges=False,
    manifold_edges=False,
)
boundary.clear_data()

In [ ]:
pl = pv.Plotter()
pl.add_mesh(
    boundary, color=theme.CONTOUR_LINE_COLOR, line_width=theme.CONTOUR_LINE_WIDTH
)

CONTOUR_LEVELS = np.linspace(-60, 60, 21)

# helpers.add_contours(plotter, source, scalars, levels, **kwargs)
helpers.add_contours(
    pl,
    mesh,
    scalars="data",
    levels=CONTOUR_LEVELS,
    cmap="RdYlBu_r",
    show_scalar_bar=True,
    scalar_bar_args=dict(title=f"", **theme.SCALAR_BAR_OPTS),
)


# box = pv.Box(mesh.bounds)
# pl.add_mesh(box, color="lightgrey", culling="front")
pl.add_floor("-z", show_edges=True, edge_color="white", color="lightgray")

helpers.add_coastlines(pl, line_width=5)
helpers.add_bounds(pl, bounds=boundary.bounds)
pl.camera.zoom(1.2)

pl.show()